In [1]:
# ____ hitting all question in dataset ____ 
import pandas as pd
import numpy as np
import os

# get path to data C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test
path = r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test'

# get list of the sub directorys in directory
sub_dir = os.listdir(path)

# create a dataframe to store the contents of the files
dataset = pd.DataFrame(columns=['problem', 'solution'])

# read in the contents of 71 files from each sub directory and print the contents
for i in sub_dir:
    # get path to sub directory
    sub_dir_path = os.path.join(path, i)
    # get list of files in sub directory
    files = os.listdir(sub_dir_path)
    # get path to each file in sub directory. only read in the first 71 files
    for j in files[:1]:
        file_path = os.path.join(sub_dir_path, j)
        # the contents of the file is a json object and i only want the 'problem' and 'solution' keys
        df = pd.read_json(file_path, orient='index', typ='series', encoding='utf-8')
        # append the contents of the file to the dataframe
        dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)

# keep only first 2 rows of dataframe
dataset = dataset[:2]


C:\Users\cayde\AppData\Local\Temp\ipykernel_9524\542045771.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_9524\542045771.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_9524\542045771.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_9524\542045771.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

In [2]:
# ____ creating a prompt for the agent to answer ____

# read in this json file C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test\algebra\1.json
df = pd.read_json(r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test\algebra\1.json', orient='index', typ='series', encoding='utf-8')

example_problem = df['problem']
print(example_problem)
# # create a prompt to send to agent. the prompt should explain that the agent is taking a test and that the agent should answer the question. and note the the equations are written in latex. the prompt should also contain the problem
# prompt = 'You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex. \n\n' + df['problem'] + '\n\n'

# print(prompt)

How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?


In [3]:
# ____ get openai api key ____
# Path: C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\gpt-4-stuff\openai_key.txt
# open the file and read in the key
with open(r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\gpt-4-stuff\openai_key.txt', 'r') as f:
    openai_api_key = f.read()


In [4]:
from __future__ import print_function
import sys

In [6]:
# ____ init agent ____
# init langchain agent with gpt-4
from langchain import LLMChain, PromptTemplate, LLMMathChain
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent, ZeroShotAgent, Tool, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.utilities import PythonREPL
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.chains import ConversationChain


llm = ChatOpenAI(model='gpt-4',temperature=0, openai_api_key=openai_api_key)
wolfram = WolframAlphaAPIWrapper(wolfram_alpha_appid='LXATGL-RJ6P2UWYT4')
python_repl = PythonREPL()

# ____ init tools ____
llm1 = OpenAI(temperature=0, openai_api_key=openai_api_key)
llm_math_chain = LLMMathChain(llm=llm1, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer simple questions about math"
    ),
    Tool(
        name="python_repl", 
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`."
    ),
    Tool(
        name="wolfram_alpha",
        func=wolfram.run,
        description="A tool that can answer questions about math, science, and more. Input should be a question. For example, `What is the derivative of x^2?`"
    )
]

# ____ init prompt ____
prefix = """You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex. You have access to the following tools:"""
suffix = """Begin! Remeber answer the math questions to the best of your ability. Equations are written in latex.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

# ____ run agent ____
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

intermediate_steps = []
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [ ]:
# test agent is working
response = agent_executor({'input':"what is 1+1?"})

# how to grab the agents intermediate steps
print(response["intermediate_steps"])

In [38]:
# ____ init gpt-4 ____
import openai

def ask_gpt4_student(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": """You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex.""",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    assistant_reply = response["choices"][0]["message"]["content"]
    return assistant_reply

def ask_gpt4_grader(question, student_answer, actual_answer):
    prompt = "question: " + question + '\nstudent_answer: ' + student_answer + '\nactual_answer: ' + actual_answer


    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": """You are evaluating a student's work. you are provided the question the student was asked, and the student's work and answer, as well as the actual answer to the question. You are to evaluate the student's work and determine if the student was CORRECT or INCORRECT. Start each response with either CORRECT or INCORRECT explain briefly why the student is right or wrong. Equations are written in latex. """,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    assistant_reply = response["choices"][0]["message"]["content"]
    return assistant_reply

In [24]:
# test gpt-4 is working
response = ask_gpt4("what is 1+1?")
print(response)

1+1=2


In [39]:
# test gpt-4 grader is working
response = ask_gpt4_grader("what is 1+1?", "1+1=2", "2")
print(response)

CORRECT: The student correctly found that 1+1 equals 2.


In [8]:
# add to 'dataset' dataframe add empty columns for agents intermediate steps and the agents answer. also add a column called gpt-4 answer.
dataset['agent_intermediate_steps'] = ''
dataset['agent_answer'] = ''
dataset['agents_score'] = ''
dataset['gpt-4_answer'] = ''
dataset['gpt-4_score'] = ''

In [40]:
# ____ run agent on dataset ____
for i in range(len(dataset)):
    # get problem from dataset
    problem = dataset['problem'][i]
    # get solution from dataset
    solution = dataset['solution'][i]
    # get the agents response
    response = agent_executor({'input':problem})
    # get the agents intermediate steps
    intermediate_steps = response["intermediate_steps"]
    # get the agents answer
    agent_answer = response["output"]
    # format agent answer and work for grading
    agent_intermediate_steps = str(dataset['agent_intermediate_steps'][i])
    agent_answer = str(dataset['agent_answer'][i])
    agent_scratchpad = agent_intermediate_steps + agent_answer
    # grade the agents answer
    agents_score = ask_gpt4_grader(problem, agent_scratchpad, solution)
    # get the gpt-4 answer
    gpt_4_answer = ask_gpt4_student(problem)
    # grade the gpt-4 answer
    gpt_4_score = ask_gpt4_grader(problem, gpt_4_answer, solution)
    # add the agents intermediate steps to the dataset
    dataset['agent_intermediate_steps'][i] = intermediate_steps
    # add the agents answer to the dataset
    dataset['agent_answer'][i] = agent_answer
    # add the gpt-4 answer to the dataset
    dataset['gpt-4_answer'][i] = gpt_4_answer
    # add the agents score to the dataset
    dataset['agents_score'][i] = agents_score
    # add the gpt-4 score to the dataset
    dataset['gpt-4_score'][i] = gpt_4_score




> Entering new AgentExecutor chain...
Thought: I need to find the vertical asymptotes of the given function. Vertical asymptotes occur when the denominator of a rational function is equal to zero.
Action: Calculator
Action Input: x^2 + x - 6 = 0

> Entering new LLMMathChain chain...
x^2 + x - 6 = 0Answer: x = 2 or x = -3
> Finished chain.

Observation: Answer: x = 2 or x = -3
Thought:I found the values of x where the denominator is zero. These are the vertical asymptotes.
Final Answer: 2

> Finished chain.


> Entering new AgentExecutor chain...
Thought: To find the probability that it will not rain tomorrow, I need to subtract the probability of rain from 1.
Action: Calculator
Action Input: 1 - 1/11

> Entering new LLMMathChain chain...
1 - 1/11
Answer: 0.9090909090909091
> Finished chain.

Observation: Answer: 0.9090909090909091
Thought:I need to convert the decimal to a common fraction.
Action: wolfram_alpha
Action Input: convert 0.9090909090909091 to a fraction
Observation: Assum

In [41]:
# ____ save dataset ____
# save dataset to cwd
dataset.to_csv('dataset.csv', index=False)